## Database pruning notebook

__Do not attempt to run this code unless you are trying to reproduce data generated from the original l2t database Feb. 2023.__

In [1]:
import pandas as pd
import numpy as np
import duckdb
import time

In [2]:
def connect_db(path: str):
    '''Runs duckdb.connect() function on database path with timing. Returns a 
duckdb.DuckDBPyConnection object'''
    
    st = time.time()

    print('Connecting to database...')
    con = duckdb.connect(path)

    et = time.time()
    elapsed_time = et - st
    print(f'Connection established! Execution time: {elapsed_time} seconds')
    return con

In [3]:
con = connect_db('/mnt/c/Users/Ryan/Documents/database')

Connecting to database...
Connection established! Execution time: 39.82907009124756 seconds


__Initial sizes__

proteins: 6 x 65427472 <br/>
protein_pairs: 21 x 181500502 <br/>
taxa: 10 x 16664 <br/>
taxa_pairs: 17 x 750633

#### First remove unnecessary columns from all 4 data tables.

In [7]:
proteins_del = ['protein_index']
protein_pairs_del = ['thermo_protein_index', 'meso_protein_index', 'local_E_value', 'query_align_start', 'query_align_end', 
                     'subject_align_start', 'subject_align_end']
taxa_del = ['filepath','ogt_scraped_string']
taxa_pairs_del = ['local_E_value', 'query_align_start', 'query_align_end', 'subject_align_start', 'subject_align_end']

tables = {'proteins':proteins_del, 'protein_pairs':protein_pairs_del, 'taxa':taxa_del, 'taxa_pairs':taxa_pairs_del}

# Drop unnecessary columns from tables
for table in tables:
    for col in tables[table]:
        cmd = f"""ALTER TABLE {table} 
        DROP COLUMN IF EXISTS {col}"""

        con.execute(cmd)
    print(f'Table {table} cleaned.')

Table proteins cleaned.
Table protein_pairs cleaned.
Table taxa cleaned.
Table taxa_pairs cleaned.


In [9]:
con.execute("""SELECT * FROM proteins LIMIT 1""").df()

,taxa_index,protein_seq,protein_desc,protein_len,protein_int_index
0,4417,MSKYDEALARMQLRSNEAFERITGQTVEQYQQEQRQNRMPTTAQLA...,hypothetical protein,4602,0


__New sizes__

proteins: 5 x 65427472 <br/>
protein_pairs: 14 x 181500502 <br/>
taxa: 8 x 16664 <br/>
taxa_pairs: 12 x 750633

#### Remove unnecessary taxa and taxa non-pairs

In [13]:
meso_cmd = """SELECT DISTINCT meso_index
FROM taxa_pairs
WHERE is_pair = True"""

useful_meso = con.execute(meso_cmd).df()

thermo_cmd = """SELECT DISTINCT thermo_index
FROM taxa_pairs
WHERE is_pair = True"""

useful_thermo = con.execute(thermo_cmd).df()
useful_taxa = tuple([i for i in useful_meso['meso_index']] + [i for i in useful_thermo['thermo_index']])

In [26]:
taxa_cmd = f"""CREATE OR REPLACE TABLE useful_taxa AS SELECT * FROM taxa WHERE taxa_index IN {useful_taxa}"""
taxa_pairs_cmd = """CREATE OR REPLACE TABLE useful_taxa_pairs AS SELECT * FROM taxa_pairs WHERE is_pair = True"""
con.execute(taxa_cmd)
con.execute(taxa_pairs_cmd)

__New sizes__

proteins: 5 x 65427472 <br/>
protein_pairs: 14 x 181500502 <br/>
taxa -> useful_taxa: 8 x 2533 <br/>
taxa_pairs -> useful taxa_pairs: 12 x 13784 <br/>

__Remove pairs with <10 C ogt difference__

In [40]:
con.execute("""CREATE TABLE ogt_taxa_pairs AS SELECT *,
taxa_m.ogt AS meso_ogt,
taxa_t.ogt AS thermo_ogt
taxa_t.ogt - taxa_m.ogt AS ogt_diff
FROM useful_taxa_pairs
JOIN useful_taxa AS taxa_m ON (useful_taxa_pairs.meso_index = taxa_m.taxa_index)
JOIN useful_taxa AS taxa_t ON (useful_taxa_pairs.thermo_index = taxa_t.taxa_index)""")

In [41]:
con.execute("""CREATE OR REPLACE TABLE wide_taxa_pairs AS SELECT * FROM ogt_taxa_pairs WHERE (thermo_ogt - meso_ogt < 10)""")

In [43]:
useful_wide_taxa_pairs = con.execute("""SELECT taxa_index FROM wide_taxa_pairs""").df()
uwtp_tuple = tuple([i for i in useful_wide_taxa_pairs['taxa_index']])

__There are only 1424 taxa pairs in this database with 10 or greater difference in growth temp.__

Let's see how many protein pairs we are now left with.

In [ ]:
final_proteins = con.execute(f"""SELECT * FROM protein_pairs WHERE taxa_pair_index IN {uwtp_tuple}""").df()
final_proteins.head()

Need to make a new notebook to demonstrate this finding as it is likely very important for the impact of the project.

- Need to show distribution of ogt differences between taxa_pairs
- Need to show loss in protein sampling frequency with increasing ogt difference
- Need to show sample size with ogt difference thresholding of multiple values